# Accidents

In [1]:
# imports
import numpy as np
import pandas as pd

In [2]:
calls_2012 = pd.read_csv('../../data/raw/Calls_for_Service_2012.csv', usecols=[0,1,6,10,12,15])
calls_2012['TimeCreate'] = pd.to_datetime(calls_2012['TimeCreate'], format='%m/%d/%Y %I:%M:%S %p')
calls_2012.set_index('NOPD_Item', inplace=True)

calls_2013 = pd.read_csv('../../data/raw/Calls_for_Service_2013.csv', usecols=[0,1,6,10,12,15])
calls_2013['TimeCreate'] = pd.to_datetime(calls_2013['TimeCreate'], format='%m/%d/%Y %I:%M:%S %p')
calls_2013.set_index('NOPD_Item', inplace=True)

calls_2014 = pd.read_csv('../../data/raw/Calls_for_Service_2014.csv', usecols=[0,1,9,13,17,20])
calls_2014['TimeCreate'] = pd.to_datetime(calls_2014['TimeCreate'], format='%m/%d/%Y %I:%M:%S %p')
calls_2014.set_index('NOPD_Item', inplace=True)

calls_2015 = pd.read_csv('../../data/raw/Calls_for_Service_2015.csv', usecols=[0,1,9,13,17,20])
calls_2015['TimeCreate'] = pd.to_datetime(calls_2015['TimeCreate'], format='%m/%d/%Y %I:%M:%S %p')
calls_2015.set_index('NOPD_Item', inplace=True)

calls_2016 = pd.read_csv('../../data/raw/Calls_for_Service_2016.csv', usecols=[0,1,9,13,17,20])
calls_2016['TimeCreate'] = pd.to_datetime(calls_2016['TimeCreate'], format='%m/%d/%Y %I:%M:%S %p')
calls_2016.set_index('NOPD_Item', inplace=True)

calls_2017 = pd.read_csv('../../data/raw/Calls_for_Service_2017.csv', usecols=[0,1,9,13,17,20])
calls_2017['TimeCreate'] = pd.to_datetime(calls_2017['TimeCreate'], format='%m/%d/%Y %I:%M:%S %p')
calls_2017.set_index('NOPD_Item', inplace=True)

calls_2018 = pd.read_csv('../../data/raw/Calls_for_Service_2018.csv', usecols=[0,1,9,13,17,20])
calls_2018['TimeCreate'] = pd.to_datetime(calls_2018['TimeCreate'], format='%m/%d/%Y %I:%M:%S %p')
calls_2018.set_index('NOPD_Item', inplace=True)

calls = pd.concat([calls_2012, calls_2013, calls_2014, calls_2015, calls_2016, calls_2017, calls_2018])

In [3]:
target_types = [
    '100',
    '100C',
    '100F',
    '100I',
    '100X',
    '20',
    '20C',
    '20F',
    '20I',
    '20X']

target_dispositions = ['NAT', 'RTF', 'GOA']
target_columns = ['TimeCreate', 'Location', 'BLOCK_ADDRESS']

In [4]:
accidents = calls[calls['Type_'].isin(target_types)]
accidents = accidents[accidents['Disposition'].isin(target_dispositions)]
accidents = accidents[target_columns]
accidents.drop_duplicates(inplace=True)

In [5]:
locations = accidents.Location.str.replace('[()]', '')
locations = locations.str.split(',')

In [6]:
lat = locations.str.get(0).astype('float')
lon = locations.str.get(1).astype('float')
accidents['lat'] = lat
accidents['lon'] = lon
accidents.drop(columns='Location', inplace=True)
accidents = accidents[accidents.lat > 1]

In [7]:
intersections = accidents[accidents.BLOCK_ADDRESS.str.contains('&', na=False)].BLOCK_ADDRESS.str.split('&')
intersection_street_1 = intersections.str.get(0).str.strip()
intersection_street_2 = intersections.str.get(1).str.strip()

In [8]:
single_street = accidents[np.logical_not(accidents.BLOCK_ADDRESS.str.contains('&', na=True))].BLOCK_ADDRESS
single_street_1 = single_street[single_street.str.split(' ', n=1).str.get(0).str.contains('X', na=False)].str.split(' ', n=1).str.get(1).str.strip()
single_street_2 = single_street[np.logical_not(single_street.str.split(' ', n=1).str.get(0).str.contains('X', na=True))].str.strip()

In [9]:
street_1 = pd.concat([intersection_street_1, single_street_1, single_street_2])

In [10]:
accidents['street_1'] = street_1
accidents['street_2'] = intersection_street_2

In [11]:
accidents.to_pickle('../../data/interim/features/accidents.pickle')